In [ ]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "unsplit")
data = ds['train']

In [ ]:
df = data.to_pandas()


In [ ]:
df

,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2
...,...,...
416804,that was what i felt when i was finally accept...,1
416805,i take every day as it comes i m just focussin...,4
416806,i just suddenly feel that everything was fake,0
416807,im feeling more eager than ever to claw back w...,1


In [ ]:
df.columns


Index(['text', 'label'], dtype='object')

In [ ]:
df.describe()


,label
count,416809.000000
mean,1.554271
std,1.490453
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    416809 non-null  object
 1   label   416809 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
df.head()

,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])


In [ ]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

dataset = {
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
}


In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 105.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=64
    )


tokenized_datasets = {
    "train": train_ds.map(tokenize_function, batched=True),
    "validation": val_ds.map(tokenize_function, batched=True),
    "test": test_ds.map(tokenize_function, batched=True)
}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/291766 [00:00<?, ? examples/s]

Map:   0%|          | 0/62521 [00:00<?, ? examples/s]

Map:   0%|          | 0/62522 [00:00<?, ? examples/s]

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6
)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)


/tmp/ipython-input-4003351878.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hessahmalmujally (hessahmalmujally-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.082800,0.091676
2,0.071300,0.094671
3,0.078800,0.096490


TrainOutput(global_step=54708, training_loss=0.10208047201659906, metrics={'train_runtime': 1716.6522, 'train_samples_per_second': 509.887, 'train_steps_per_second': 31.869, 'total_flos': 1.4494590026592768e+16, 'train_loss': 0.10208047201659906, 'epoch': 3.0})

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
from evaluate import load
import numpy as np

metric = load("accuracy")
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=1)
accuracy = metric.compute(predictions=preds, references=predictions.label_ids)
print(f"Test Accuracy: {accuracy['accuracy']:.4f}")


Test Accuracy: 0.9415


In [26]:
trainer.save_model("/.../.../.../emotion-model-byHasoosy")
tokenizer.save_pretrained("/.../.../.../emotion-model-byHasoosy")


('/content/drive/MyDrive/emotion-model-byHasoosy/tokenizer_config.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/special_tokens_map.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/vocab.txt',
 '/content/drive/MyDrive/emotion-model-byHasoosy/added_tokens.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/tokenizer.json')

In [27]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


tokenizer.save_pretrained("/.../.../.../emotion-model-byHasoosy")


('/content/drive/MyDrive/emotion-model-byHasoosy/tokenizer_config.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/special_tokens_map.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/vocab.txt',
 '/content/drive/MyDrive/emotion-model-byHasoosy/added_tokens.json',
 '/content/drive/MyDrive/emotion-model-byHasoosy/tokenizer.json')

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "/.../.../.../emotion-model-byHasoosy"


tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
    return label_names[prediction]


print(predict_emotion("I LOST MY MOM "))


sadness


In [30]:
predict_emotion("I feel so lonely and scared")


'sadness'

In [33]:
!pip install diffusers transformers accelerate safetensors --upgrade


In [2]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
prompt = """A child celebrating and happy and seeing fireworks in the sky
"""
image = pipe(prompt).images[0]
image.save("output.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image


emotion_model_path = "/.../.../.../emotion-model-byHasoosy"
tokenizer = AutoTokenizer.from_pretrained(emotion_model_path)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_path)


def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = emotion_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
    return label_names[prediction]


sd_model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(sd_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")


def generate_emotional_art(user_input):
    emotion = predict_emotion(user_input)
    final_prompt = f"A childlike drawing representing the feeling of {emotion}, inspired by the sentence: '{user_input}'. Use pastel colors and simple style"
    image = pipe(final_prompt).images[0]
    image.save("output.png")
    return image, emotion


user_input = "A child celebrating and happy and seeing fireworks in the sky"
image, predicted_emotion = generate_emotional_art(user_input)


print(f"Predicted Emotion: {predicted_emotion}")
image.show()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Predicted Emotion: joy


In [17]:
import gradio as gr

def generate_with_gradio(user_input):
    image, emotion = generate_emotional_art(user_input)
    return image, emotion

interface = gr.Interface(
    fn=generate_with_gradio,
    inputs=gr.Textbox(label="Write something from the child's perspective"),
    outputs=[gr.Image(label="Generated Art"), gr.Text(label="Detected Emotion")],
    title="Draw My Emotion 🎨",
    description="The AI reads what the child writes and expresses it through an emotional drawing."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc247ef6feb09a421e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [28]:
import gradio as gr

def generate_with_gradio(user_input):
    image, emotion = generate_emotional_art(user_input)
    return image, emotion

interface = gr.Interface(
    fn=generate_with_gradio,
    inputs=gr.Textbox(label="Write something from the child's perspective"),
    outputs=[gr.Image(label="Generated Art"), gr.Text(label="Detected Emotion")],
    title="Draw My Emotion 🎨",
    description="The AI reads what the child writes and expresses it through an emotional drawing."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c0915ee988d6398f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
